# Datasets

Run this `gcsfuse` cell if you can't list the folders inside of "/gcs"

In [2]:
!gcsfuse --implicit-dirs "~/gcs"

I1109 16:04:22.597573 2023/11/09 16:04:22.597530 Start gcsfuse/0.42.5 (Go version go1.20.3) for app "" using mount point: /home/jupyter/gcs


When using GCS buckets, use "/gcs" instead of "gs://"

In [3]:
#from sklearn.model_selection import train_test_split

dataset_path = "/home/jupyter/gcs/serena-shsw-datasets"
training_dataset = dataset_path + "/FER-2013/train"
test_dataset = dataset_path + "/FER-2013/test"

# Split the dataset into training and validation sets
#training_dataset, validation_dataset = train_test_split(training_dataset, test_size=0.2, random_state=42)

# Output directory contents
!echo "Train"
!ls {training_dataset}
!echo "Test"
!ls {test_dataset}
!echo "Valid"
!ls{validation_dataset}

Train
angry  disgust	fear  happy  neutral  sad  surprise
Test
angry  disgust	fear  happy  neutral  sad  surprise
Valid
/bin/bash: ls{validation_dataset}: command not found


# Import Library

In [4]:
import csv
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

2023-11-09 16:04:42.882299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 16:04:44.332522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-09 16:04:44.332563: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-09 16:04:47.959594: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

# Create Model

In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

2023-11-09 16:04:50.452771: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-09 16:04:50.452813: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-09 16:04:50.452844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (emotion-detector-nb): /proc/driver/nvidia/version does not exist
2023-11-09 16:04:50.453756: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
num_classes = 7  # Gantikan dengan jumlah sebenarnya dari kelas emosi dalam dataset Anda
batch_size = 32 * 4 # 128 seems to be ideal
num_epochs = 10
train_data_dir = 'path_to_train_data_directory'
validation_data_dir = 'path_to_validation_data_directory'
test_data_dir = 'path_to_test_data_directory'

In [7]:
#num_classes = 7  # Gantikan dengan jumlah sebenarnya dari kelas emosi dalam dataset Anda

#x = layers.GlobalAveragePooling2D()(x)
#x = layers.Dense(1024, activation='relu')(x)
#predictions = layers.Dense(num_classes, activation='softmax')(x)

#model = models.Model(inputs=base_model.input, outputs=predictions)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

# Data Augmentation

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    directory=training_dataset,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.


# Data Generating

In [10]:
validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

validation_generator = validation_datagen.flow_from_directory(
    directory=test_dataset,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 7178 images belonging to 7 classes.


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10, 
    validation_data=validation_generator,
)

In [12]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
)

Epoch 1/10


KeyboardInterrupt: 

# Saving Model

Vertex AI expects the model artifacts to be saved in `BASE_OUTPUT_DIRECTORY/model/` when you want to train a new version of a model

In [13]:
saved_model_path = dataset_path + "/models/serena-emotion-detector/model"

In [14]:
model.save(saved_model_path)

INFO:tensorflow:Assets written to: /home/jupyter/gcs/serena-shsw-datasets/models/serena-emotion-detector/assets


INFO:tensorflow:Assets written to: /home/jupyter/gcs/serena-shsw-datasets/models/serena-emotion-detector/assets


# Evaluate Model

In [15]:
saved_emotion_detector = tf.keras.models.load_model(saved_model_path)

# Check its architecture
saved_emotion_detector.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 14,847,815
Trainable params: 133,127
Non-trainable params: 14,714,688
______________________________________

In [17]:
val_loss, val_accuracy = model.evaluate(training_datasets, training_labels)
print(f'Validation Accuracy: {val_accuracy}')

test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f'Test Accuracy: {test_accuracy}')

NameError: name 'training_datasets' is not defined